In [1]:
import os, sys, time
from time import sleep
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import fnmatch, glob, shutil
import tempfile, logging
from zipfile import ZipFile
from PyPDF2 import PdfFileWriter, PdfFileReader

In [2]:
class CustomCrate(): # {
    
    out_directory = "."
    watermark = "C:/data/inbound/Agilent_CofA_Letterhead_03-21-19.pdf"
    
    def __init__(self, the_date, list_of_directories, in_file): # {
        self.the_date = the_date
        self.list_of_directories = list_of_directories
        self.in_file = str(in_file)
        # CREATE ZIP FOLDER 
        self.zip_dir_path = os.path.join(self.out_directory, self.the_date)
        # if 'zip_folder' DOES NOT YET EXIST!
        if not os.path.exists(self.zip_dir_path): #{
            # MAKE IT EXIST!
            os.makedirs(self.zip_dir_path)
        #}
        # RESET/RE-ESTABLISH 'out_directory'
        self.out_directory = Path(self.zip_dir_path)
        # CALL FUNCTION TO CREATE DATE IDX
        self.df_index = self.create_date_idx()
    # }
    
    def create_watermark(self, input_pdf, output, watermark): # {
        # TRY THE FOLLOWING
        try: #{
            watermark_obj = PdfFileReader(watermark)
            watermark_page = watermark_obj.getPage(0)
            
            pdf_reader = PdfFileReader(input_pdf)
            pdf_writer = PdfFileWriter()
            
            # Watermark all the pages
            for page in range(pdf_reader.getNumPages()): #{
                page = pdf_reader.getPage(page)
                page.mergePage(watermark_page)
                pdf_writer.addPage(page)
            #}
            
            with open(output, 'wb') as out: #{
                pdf_writer.write(out)
            #}
        #}
        except: #{
            errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
            print("\n" + typeE +
                  "\n" + fileE +
                  "\n" + lineE +
                  "\n" + messageE)
        #}
        else: #{
            print("[Watermark-PDF] FIN...")
        #}
        return
    # }
    
    def create_date_idx(self): # {
        # TRY THE FOLLOWING
        try: # {
            ###########################################################
            # CREATE EMPTY DATAFRAME TO HOLD FILE NAMES AND DATE INDEX
            cols = ['Directory', 'Filename', 'Creation', 'Lot_No']
            df_index = pd.DataFrame(data=None, columns=cols, dtype=np.str)
            # READIN .CSV
            df_batch = pd.read_csv(self.in_file, header=None, 
                                   names=['Lot_No'], dtype=np.str, engine='python'
                                  )
            ###########################################################
            # FOR EACH PATH LISTED:
            for directory in self.list_of_directories: # {
                # SET "in_directory" to be one from LIST
                in_directory = Path(directory)
                print(in_directory)
                print("==============================\n")
                # EMPTY LISTS TO FILL WITH DATA
                filepath_col = [] # for FILE DIRECTORY
                filename_col = [] # for FILE NAME
                filedate_col = [] # creation DATE
                lot_num_col = [] # FOR CURRENT LOT #
                # CREATE REGEX VAR FOR DIR
                test_glob = str(directory + "*")
                print(test_glob)
                # COUNTER
                x = 0
                # CHECK IF THERE ARE ANY SUB-FOLDERS (via glob)
                sub_folder_num = len(sorted(glob.glob(directory + "*")))
                print("# of sub-diretories: " + str(sub_folder_num))
                if sub_folder_num == 0: # {
                    print("GLOB AGAIN")
                    # GLOB / ITERATE / LOOP AGAIN
                    sleep(10)
                # }
                else: # {
                    print("NO SUB-DIRS! ")
                    sleep(10)
                # }
                for lot_no in df_batch.itertuples(): # {
                    print("<< LOT #: " + str(lot_no[1]) + " >>")
                    # GLOB-ITERATE THRU EACH FILE MATCHING NAME ETC
                    for name in sorted(glob.glob(directory
                                                 + "*"
                                                 + str(lot_no[1])
                                                 + "*")): # {
                        # CREATE PATH VARIABLE
                        pdf_path = Path(str(name))
                        print("PDF:\n\t" + str(pdf_path))
                        print("exists: " + str(os.path.exists(pdf_path)))
                        # GET DIR NAME
                        dir_name = os.path.dirname(pdf_path)
                        print("DIR_NAME:\n\t" + str(dir_name))
                        # GET BASE NAME (FILE NAME)
                        file_name = os.path.basename(pdf_path)
                        print("FILE_NAME:\n\t" + str(file_name))
                        # GET FILE CREATION DATE
                        file_date = self.pull_creation_timestamp(pdf_path)
                        print("FILE_DATE:\n\t" + str(file_date))
                        #####################
                        # FILL LIST COLUMNS #
                        #####################
                        filepath_col.append(str(dir_name))
                        filename_col.append(str(file_name))
                        filedate_col.append(str(file_date))
                        lot_num_col.append(str(lot_no[1]))
                        # INCREASE COUNT
                        x += x
                    # }
                    """
                    >>> IF NOT BY "most_Recent"... 
                    >>> CREATE SEPERATE DATAFRAME <<<
                    >>> PER INDIV. FILE FOUND HERE <<<
                    """
                    # CREATE [LOT #] DATAFRAME
                    # [2020-03-05]\\df_lot_no = pd.DataFrame(data=None, dtype=np.str)
                # }
                # Create DataFrame???
                df_test = pd.DataFrame(data=None, dtype=np.str)
                # Create Columns??
                df_test['Directory'] = filepath_col
                df_test['Filename'] = filename_col
                df_test['Creation'] = filedate_col
                df_test['Lot_No'] = lot_num_col
                # SET INDEX OF DATAFRAME
                df_test.set_index(['Creation'], inplace=True)
                # SORT INDEX?
                df_test.sort_index(inplace=True)
                # EXPORT?
                df_test.to_csv("df_test_" + str(directory)[:1] + "_.csv", index=True)
                # CONCAT/CREATE "result"
                result = pd.concat([df_index, df_test])
                # APPEND TO MAIN DATAFRAME
                df_index = df_index.append(result)
            # }
            
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
            print("\n" + typeE +
                  "\n" + fileE +
                  "\n" + lineE +
                  "\n" + messageE)
        # }
        else: # {
            print("Operation Completed Successfully...")
        # }
    # }
    
    def pull_creation_timestamp(self, a_file_path): # {
        # TRY THE FOLLOWING
        try: # {
            # FORCE PATH VARIABLE
            the_path = Path(str(a_file_path))
            # GET MODIFIED TIME
            mtime = os.path.getmtime(the_path)
            # GET CREATE TIME
            ctime = os.path.getctime(the_path)
            # CREATE DATE VAR
            if ctime < mtime: # {
                # FORMAT DATE VAR AS str
                date_str = str(datetime.fromtimestamp(ctime))
            # }
            # ELSE.... MODIFIED TIME IS OLDER..
            else: # {
                # FORMAT DATE VAR as str
                date_str = str(datetime.fromtimestamp(mtime))
            # }
        # }
        except: # {
            errorMessage = str(sys.exc_info()[0]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n\t\t"
            errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            typeE = str("TYPE : " + str(exc_type))
            fileE = str("FILE : " + str(fname))
            lineE = str("LINE : " + str(exc_tb.tb_lineno))
            messageE = str("MESG : " + "\n" + str(errorMessage) + "\n")
            logging.error("\n" + typeE +
                          "\n" + fileE +
                          "\n" + lineE +
                          "\n" + messageE)
        #}
        else: # {
            print("Operation Completed Successfully...")
            return date_str
        # }
        finally: # {
            print("FIN...")
        # }
        
    # }
    
# }

In [3]:
if __name__ == "__main__": # {
    # CREATE TIMESTAMP FOR ZIP FOLDER
    ts = pd.Timestamp.now()
    ts_str = str(ts)[:10]
    print(ts_str)
    directory_list = ['F:/APPS/CofA/',
                      "G:/C of A's/Agilent/",
                      "J:/controlled_docs/SDS/Agilent_SDS/USA/"]
    # CREATE / CALL MAIN CLASS FUNCTION
    create_crate = CustomCrate(the_date=ts_str,
                              list_of_directories=directory_list,
                              in_file='2020-03-05-batch-list.csv'
                              )
# }

2020-03-05
F:\APPS\CofA

F:/APPS/CofA/*
# of sub-diretories: 67125
GLOB AGAIN
<< LOT #: IAA-213 >>
PDF:
	F:\APPS\CofA\IAA-213@0006501009.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-213@0006501009.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2019-10-31 13:57:21.332925
PDF:
	F:\APPS\CofA\IAA-213@J00295.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-213@J00295.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-25 09:52:00.824611
PDF:
	F:\APPS\CofA\IAA-213@J00695.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-213@J00695.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-25 09:53:51.214530
PDF:
	F:\APPS\CofA\IAA-213@J00775.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-213@J00775.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-25 09:54:27.917420
PDF:
	F:\APPS\CofA\IAA-213@M01080.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-213@M01080.pdf
Operation Completed Successfully...

PDF:
	F:\APPS\CofA\IAA-251@CL-3448.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-251@CL-3448.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-26 13:29:50.031748
PDF:
	F:\APPS\CofA\IAA-251@CS-6314.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-251@CS-6314.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2019-01-02 14:01:55.617777
PDF:
	F:\APPS\CofA\IAA-251@J00297.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-251@J00297.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-26 13:30:58.875057
PDF:
	F:\APPS\CofA\IAA-251@J01034.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-251@J01034.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-26 13:31:56.390314
PDF:
	F:\APPS\CofA\IAA-251@K00435.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-251@K00435.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-26 13:32:40.546282
PDF:
	F:\APPS\CofA\IAA-251@L01286.pdf
exists: True
DIR_N

<< LOT #: ICP-351 >>
PDF:
	F:\APPS\CofA\ICP-351@AU-00032.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-351@AU-00032.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-30 07:46:26.421000
PDF:
	F:\APPS\CofA\ICP-351@AU-00901.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-351@AU-00901.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-09-03 16:13:33.577463
PDF:
	F:\APPS\CofA\ICP-351@AU-01006.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-351@AU-01006.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2016-05-20 14:06:03.028953
<< LOT #: ICP-451 >>
PDF:
	F:\APPS\CofA\ICP-451@0006517000.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-451@0006517000.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2020-02-13 15:55:42.033111
PDF:
	F:\APPS\CofA\ICP-451@AU-00065.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-451@AU-00065.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-10-08 10:22:41.6

exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-233@R00893.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-25 16:37:04.455149
<< LOT #: IAA-033 >>
PDF:
	F:\APPS\CofA\IAA-033@E00346Z.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-033@E00346Z.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-24 10:43:41.140909
PDF:
	F:\APPS\CofA\IAA-033@L00810.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-033@L00810.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-24 10:45:05.218496
PDF:
	F:\APPS\CofA\IAA-033@M00612.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-033@M00612.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-24 10:46:05.780608
PDF:
	F:\APPS\CofA\IAA-033@M00612A.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-033@M00612A.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2019-04-04 09:16:20.064524
PDF:
	F:\APPS\CofA\IAA-033@M01086.pdf
exists: True
DIR_NAME:
	F:\APPS\Cof

PDF:
	F:\APPS\CofA\IAA-004@J01222.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-004@J01222.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-19 11:10:31.404126
PDF:
	F:\APPS\CofA\IAA-004@L01180.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-004@L01180.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-19 11:11:14.497601
PDF:
	F:\APPS\CofA\IAA-004@N082599.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-004@N082599.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2017-06-05 06:46:20.123799
PDF:
	F:\APPS\CofA\IAA-004@R00171.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-004@R00171.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-19 11:09:31.857632
<< LOT #: ICP-004 >>
PDF:
	F:\APPS\CofA\ICP-004@0006479275.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-004@0006479275.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2019-07-30 07:43:39.791299
PDF:
	F:\APPS\CofA\ICP-004@CM

...

	2017-01-31 10:57:11.580651
PDF:
	F:\APPS\CofA\IAA-282@N082724.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-282@N082724.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2017-07-27 12:00:46.940862
PDF:
	F:\APPS\CofA\IAA-282@N083460.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-282@N083460.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2018-08-13 12:22:42.073462
PDF:
	F:\APPS\CofA\IAA-282@T00363.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-282@T00363.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-26 15:41:07.715705
<< LOT #: IAA-082 >>
PDF:
	F:\APPS\CofA\IAA-082@H00846A.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-082@H00846A.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-24 16:13:25.710194
PDF:
	F:\APPS\CofA\IAA-082@L00244.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-082@L00244.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-24 16:14:41.428459
PDF:


PDF:
	F:\APPS\CofA\ICP-003@0006480652.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-003@0006480652.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2019-07-30 08:00:23.179142
PDF:
	F:\APPS\CofA\ICP-003@CL-2953.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-003@CL-2953.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-04-02 15:11:05.370665
PDF:
	F:\APPS\CofA\ICP-003@CM-3894.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-003@CM-3894.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-07-17 14:41:15.800329
PDF:
	F:\APPS\CofA\ICP-003@CP-5867.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-003@CP-5867.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2016-12-05 07:00:02.644849
PDF:
	F:\APPS\CofA\ICP-003@CR-3774.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-003@CR-3774.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2017-08-21 13:28:17.848263
PDF:
	F:\APPS\CofA\ICP-003@G00921A.pdf
exist

PDF:
	F:\APPS\CofA\IAA-012@E00362Z.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-012@E00362Z.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-19 11:33:53.997650
PDF:
	F:\APPS\CofA\IAA-012@L00345.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-012@L00345.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-19 11:35:20.211285
PDF:
	F:\APPS\CofA\IAA-012@N081669.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-012@N081669.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2016-02-03 13:52:52.441467
PDF:
	F:\APPS\CofA\IAA-012@N081857.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-012@N081857.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2016-05-04 15:56:47.568369
PDF:
	F:\APPS\CofA\IAA-012@N083422.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	IAA-012@N083422.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2018-07-18 08:38:33.421586
PDF:
	F:\APPS\CofA\IAA-012@R00244.pdf
exists: True
D

PDF:
	F:\APPS\CofA\ICP-025@0006480816.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-025@0006480816.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2019-07-31 13:19:45.852840
PDF:
	F:\APPS\CofA\ICP-025@CL-3978.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-025@CL-3978.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-16 12:19:15.978311
PDF:
	F:\APPS\CofA\ICP-025@CP-1494.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-025@CP-1494.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2016-03-31 13:32:51.335876
PDF:
	F:\APPS\CofA\ICP-025@CP-5606.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-025@CP-5606.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2016-11-10 14:23:29.128172
PDF:
	F:\APPS\CofA\ICP-025@CR-2126.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-025@CR-2126.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2017-05-10 11:57:11.286370
PDF:
	F:\APPS\CofA\ICP-025@IH-0142.pdf
exist

Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-04-22 09:26:21.618061
PDF:
	F:\APPS\CofA\ICP-080@P00016.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-080@P00016.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-04-22 09:29:24.946186
PDF:
	F:\APPS\CofA\ICP-080@P00139.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-080@P00139.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-04-22 09:30:53.039936
PDF:
	F:\APPS\CofA\ICP-080@T00602.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-080@T00602.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-03-16 12:01:31.878202
<< LOT #: ICP-080-25 >>
<< LOT #: ICP-180 >>
PDF:
	F:\APPS\CofA\ICP-180@CM-4459.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-180@CM-4459.pdf
Operation Completed Successfully...
FIN...
FILE_DATE:
	2015-08-20 06:45:16.819007
PDF:
	F:\APPS\CofA\ICP-180@CS-3166.pdf
exists: True
DIR_NAME:
	F:\APPS\CofA
FILE_NAME:
	ICP-180@CS-3166.pdf
Operation C